# CNN Using Ayurvethic Plant Idenification

## Packages and Module configuations

### Nessary Packages

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import pandas as pd

### Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Model Related Packages

In [28]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

##  Cofigurations

### Dataset

In [4]:
dataset_path = '/content/drive/MyDrive/AYUSHVISION/preTrained/final'

In [17]:
# Define the path to the train folder containing images of dogs and cats
train_data_dir,validation_data_dir,test_data_dir,predict_data_dir = map(lambda fold:os.path.join(dataset_path,fold),('train','val','test','predicts'))
target_size=(224,224)
batch_size=32
class_mode='categorical'


### Model related Config

In [18]:
input_shape=list(target_size)
input_shape.append(3)
input_shape=tuple(input_shape)

In [20]:
class_count=32

In [24]:
dropout_rate=0.3

## Training

In [33]:
epochs=20
lr=1e-4
early_stopping_patience=10
metrics=["accuracy"]
lossfn='categorical_crossentropy'

## Preprocessing

### Augumentaion

In [15]:
train_datagen = image.ImageDataGenerator(rescale=1./255,rotation_range=35,width_shift_range=0.3,brightness_range=(0.1,0.4))
validation_datagen = image.ImageDataGenerator(rescale=1./255,rotation_range=30,width_shift_range=0.3,brightness_range=(0.1,0.4))
test_datagen = image.ImageDataGenerator(rescale=1./255,rotation_range=45)

In [16]:
# Load and augment the training dataset
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size
)

# Load and augment the validating dataset
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size
)

# Load and augment the test dataset
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size
)

Found 11174 images belonging to 32 classes.
Found 1598 images belonging to 32 classes.
Found 3197 images belonging to 32 classes.


In [26]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(dropout_rate))
model.add(Conv2D(254, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(class_count, activation='softmax'))

In [31]:
# compile model
optimizer = Adam(learning_rate=lr)

In [34]:
model.compile(optimizer=optimizer, loss=lossfn, metrics=metrics)

## Training

In [32]:
# checkpoint = ModelCheckpoint(f'{working_dir}/efficentnet/{epochs}best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True)

In [ ]:
model_history=model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[early_stopping]
    )

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


121/350 ━━━━━━━━━━━━━━━━━━━━ 36:54 10s/step - accuracy: 0.0307 - loss: 3.4701

In [ ]:

# Assuming model_history is the result of model.fit()
history = model_history.history

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot Training and Validation Accuracy
axs[0].plot(history['accuracy'], label='Training Accuracy')
axs[0].plot(history['val_accuracy'], label='Validation Accuracy')
axs[0].legend()
axs[0].set_title("Training and Validation Accuracy")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Accuracy")

# Plot Training and Validation Loss
axs[1].plot(history['loss'], label='Training Loss')
axs[1].plot(history['val_loss'], label='Validation Loss')
axs[1].legend()
axs[1].set_title("Training and Validation Loss")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Loss")

# Adjust layout
plt.tight_layout()
plt.show()


## Test

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')
print(f'Test Loss: {test_loss*100:.2f}%')

In [ ]:
# Get predictions
predictions = model.predict(test_ds_gen)

# Unpack test images and labels
test_images, test_labels = next(iter(test_ds_gen))  # Assuming test_ds_gen is a batched dataset

# Define the number of images to display
num_images = 10
plt.figure(figsize=(15, 6))

for i in range(num_images):
    plt.subplot(2, 5, i + 1)  # Create a grid for displaying images
    plt.imshow(test_images[i])  # Display the test image
    plt.axis('off')  # Remove axis for better display

    # Get predicted and actual labels
    predicted_label = np.argmax(predictions[i])  # Get the index of the highest probability
    actual_label = test_labels[i].numpy() if hasattr(test_labels[i], 'numpy') else test_labels[i]

    # Display prediction correctness
    is_correct = predicted_label == actual_label
    color = "green" if is_correct else "red"
    plt.title(f"Pred: {predicted_label}\nActual: {actual_label}", color=color)

plt.tight_layout()
plt.show()


## Predications

## Refrences


1.   [Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model)
3.   [Matplotlib](https://matplotlib.org/)
2.   [Imagegenrator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)


